#Formatter

mock-версия пайплайна

Зависимости

In [5]:
#!pip install -q stanza langdetect deep-translator sentence-transformers scikit-learn langchain langchain-openai python-dotenv stop-words

импорты

In [ ]:
# from langdetect import detect, LangDetectException
# from deep_translator import GoogleTranslator
# from collections import Counter
# import stanza
# from stop_words import get_stop_words
# #from .config import USE_GPU
# import os
# import hashlib
# from collections import defaultdict
# from typing import List, Dict, Any, Optional
# from dotenv import load_dotenv
# from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import JsonOutputParser
# from pydantic import BaseModel, Field
# from langdetect import detect, LangDetectException
# from deep_translator import GoogleTranslator
# from collections import Counter
# import stanza
# from stop_words import get_stop_words
# import numpy as np
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering

In [2]:
#import stanza
#stanza.download('ru')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


## Основные классы

In [11]:
# mock_classes

class NewsProcessor:
    def process_news_list(self, news_list):
        """Обогащает поля с новостями"""
        for item in news_list:
            item["translation"] = item.get("text", "")
            item["entities"] = ["Сбербанк", "ЦБ РФ"]  # фиксированные сущности
            item["keywords"] = ["ставка", "инфляция", "рынок"]  # заглушка
            item["lemmatized"] = item["text"].lower()
        return news_list


class NewsClusterer:
    def cluster(self, news_list, **kwargs):
        """Присваивает всем новостям кластер c аггломеративной кластеризацией"""
        for i, item in enumerate(news_list):
            # Можно имитировать разные кластеры, но для простоты — один
            item["cluster_id"] = 0
        return news_list


class DraftGenerator:
    def generate_drafts(self, news_list):
        """Генерирует черновик на основе кластера."""
        if not news_list:
            return {"drafts": []}

        # Имитируем один кластер
        draft = {
            "title": "Ключевая ставка ЦБ РФ повышена до 17%",
            "draft": {
                "lead": "Банк России усилил денежно-кредитную политику на фоне роста инфляционных рисков.",
                "bullet_1": "Контекст: Решение принято на фоне устойчивого превышения инфляции над целевым уровнем.",
                "bullet_2": "Сценарий: Ожидается замедление кредитования и рост доходности ОФЗ.",
                "bullet_3": "Рекомендация: Инвесторам — увеличить долю в коротких облигациях и защищённых акциях.",
                "quote": "«Мы видим необходимость в более жёсткой политике», — заявил председатель ЦБ."
            },
            "sources": [
                {
                    "name": item["source"]["name"],
                    "url": item["source"]["url"],
                    "credibility": item["source"]["credibility"]
                }
                for item in news_list[:2]  # первые 2 источника
            ],
            "hotness": max(n["hotness"] for n in news_list),
            "entities": ["ЦБ РФ", "ключевая ставка"],
            "cluster_id": 0
        }
        return {"drafts": [draft]}

In [12]:
# import os
# from google.colab import userdata

In [19]:
#os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')

In [14]:
# Пример входных данных (замените на свои)
raw_news = [
    {
        "news": {
            "title": "ЦБ РФ повысил ключевую ставку",
            "content": "Банк России на заседании 27 сентября 2024 года принял решение повысить ключевую ставку на 50 базисных пунктов — до 17% годовых.",
            "source": "Коммерсантъ",
            "link": "https://example.com/cbr-rate-hike"
        },
        "validation": {"credibility_score": 9},
        "hotness_score": 8
    },
    {
        "news": {
            "title": "Ключевая ставка ЦБ выросла до 17%",
            "content": "Регулятор усилил денежно-кредитную политику из-за роста инфляционных ожиданий.",
            "source": "РИА Новости",
            "link": "https://example.com/ria-cbr"
        },
        "validation": {"credibility_score": 8},
        "hotness_score": 7
    }
]

## Пайплайн

In [15]:
def run_pipeline(news):
    processor = NewsProcessor()
    enriched = processor.process_news_list(news)
    clusterer = NewsClusterer()
    clustered = clusterer.cluster(enriched)
    generator = DraftGenerator()
    drafts = generator.generate_drafts(clustered)
    return drafts

# Запуск
result = run_pipeline(news)

## Результат

In [17]:
import json
print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "drafts": [
    {
      "title": "Ключевая ставка ЦБ РФ повышена до 17%",
      "draft": {
        "lead": "Банк России усилил денежно-кредитную политику на фоне роста инфляционных рисков.",
        "bullet_1": "Контекст: Решение принято на фоне устойчивого превышения инфляции над целевым уровнем.",
        "bullet_2": "Сценарий: Ожидается замедление кредитования и рост доходности ОФЗ.",
        "bullet_3": "Рекомендация: Инвесторам — увеличить долю в коротких облигациях и защищённых акциях.",
        "quote": "«Мы видим необходимость в более жёсткой политике», — заявил председатель ЦБ."
      },
      "sources": [
        {
          "name": "Коммерсантъ",
          "url": "https://example.com/cbr-rate-hike",
          "credibility": 0.9
        },
        {
          "name": "РИА Новости",
          "url": "https://example.com/ria-cbr",
          "credibility": 0.8
        }
      ],
      "hotness": 0.8,
      "entities": [
        "ЦБ РФ",
        "ключевая ставка"
      ],
   